In [1]:
import pandas as pd
from assign_unique_ids_functions import *
import inspect
import itertools
import numpy as np

In [2]:
def intersect(a, b):
     return list(set(a) & set(b))
def setdiff(a,b):
    return list(set(a) - set(b))
def union(a,b):
    return list(set(a) | set(b))

In [3]:
asd = pd.read_csv("input/all-sworn_demographics.csv.gz")
asud = pd.read_csv("input/all-sworn-units_demographics.csv.gz")
print(asd.sort_values('Last.Name').head())
print(asd.sort_values('Last.Name').head())

   all_sworn_ID First.Name Last.Name Suffix.Name Appointed.Date  Birth.Year  \
0         12735    JEFFERY     AARON         NaN     2005-09-26        1971   
1         16413     KARINA     AARON         NaN     2005-09-26        1980   
2          4767     DANIEL     ABATE         NaN     1970-06-15        1942   
3          1207    ANTHONY    ABBATE         NaN     1994-12-05        1968   
4          3134     CARMEL    ABBATE         NaN     1969-01-06        1942   

   Gender      Race Middle.Initial  
0    MALE     WHITE              M  
1  FEMALE  HISPANIC            NaN  
2    MALE     WHITE              P  
3    MALE     WHITE              G  
4    MALE     WHITE              G  
   all_sworn_ID First.Name Last.Name Suffix.Name Appointed.Date  Birth.Year  \
0         12735    JEFFERY     AARON         NaN     2005-09-26        1971   
1         16413     KARINA     AARON         NaN     2005-09-26        1980   
2          4767     DANIEL     ABATE         NaN     1970-06-15   

In [4]:
asd[asd['Last.Name'].str.contains('JUR')].sort_values('First.Name')

all_sworn_ID First.Name     Last.Name Suffix.Name Appointed.Date  \
14049         31996     AUDREY  JURCZYKOWSKI         NaN     1999-05-10   
14058          6213     DENNIS     JURKOWSKI         NaN     1967-11-13   
14053          9357        GEO     JURGENSEN         NaN     1953-09-21   
14055          9500     GEORGE        JURICH         NaN     1958-05-01   
14059         11761      JAMES         JURTA         NaN     1968-02-26   
14051         14038       JOHN         JUREK         NaN     1995-10-23   
14056         14039       JOHN        JURISS         NaN     1956-05-16   
14047         19105     MARINA       JURASSI         NaN     2006-08-28   
7040          21704    NEBOJSA    DJURDJEVIC         NaN     2014-02-18   
14052         24449    RICHARD         JUREK         NaN     1962-03-12   
14048         25381     ROBERT     JURCZENKO         NaN     1966-11-28   
14050         25382     ROBERT  JURCZYKOWSKI         NaN     1995-06-05   
14057         26297      ROMAN    JURKIEWICZ         NaN     1972-11-20   
14054         29215     THOMAS         JURIC         NaN     1973-02-19   
14046         30326     VICTOR        JURADO         NaN     2016-06-27   

       Birth.Year  Gender      Race Middle.Initial  
14049        1970  FEMALE     WHITE              A  
14058        1946    MALE     WHITE              A  
14053        1920    MALE     WHITE              O  
14055        1932    MALE     WHITE              P  
14059        1947    MALE     WHITE              J  
14051        1973    MALE     WHITE              J  
14056        1931    MALE     WHITE              J  
14047        1979  FEMALE     WHITE              L  
7040         1984    MALE     WHITE            NaN  
14052        1938    MALE     WHITE              R  
14048        1936    MALE     WHITE            NaN  
14050        1971    MALE     WHITE            NaN  
14057        1948    MALE     WHITE            NaN  
14054        1945    MALE     WHITE              M  
14046        1987    MALE  HISPANIC              A

In [5]:
def take_first_four(x):
    return x[:4]


def BY_to_CA(x):
    return 2016 - x


def add_columns(df, add_cols = ["F4FN", "F4LN", "Current.Age", "BY_to_CA"]):
    if "F4FN" in add_cols and "First.Name" in df.columns:
            df['F4FN'] = df['First.Name'].map(take_first_four)
    if "F4LN" in add_cols and 'Last.Name' in df.columns:
            df['F4LN'] = df['Last.Name'].map(take_first_four)
    if "Current.Age" in add_cols and "Current.Age" in df.columns:
        df['Current.Age.p1'] = df['Current.Age']
        df['Current.Age.m1'] = df['Current.Age']
    if "BY_to_CA" in add_cols and "Birth.Year" in df.columns:
        by_to_ca = lambda x: 2016 - x
        df['Current.Age.p1'] = df['Birth.Year'].map(BY_to_CA)
        df['Current.Age.m1'] = df['Birth.Year'].map(BY_to_CA) - 1  
    return df

In [6]:
def generate_on_lists(data_cols, base_lists):
    merge_list = []
    
    for col_list in base_lists:
        if intersect(col_list, data_cols):
            ml = intersect(col_list, data_cols)
            if '' in col_list:
                ml.append('')
            merge_list.append(sorted(ml, reverse=True))
            
    merge_list = list(itertools.product(*reversed(merge_list)))
    merge_list = [[i for i in ml if i != ''] for ml in merge_list]
    
    return merge_list

In [7]:
def loop_merge(df1, df2, on_lists, keep_columns, return_unmatched = True):
    dfm = pd.DataFrame(columns = keep_columns + ['Match'])
    for mc in on_lists:
        df1t = remove_duplicates(df1[keep_columns[:1] + mc], mc)
        df2t = remove_duplicates(df2[keep_columns[1:] + mc], mc)
        dfmt = df1t.merge(df2t, on=mc, how='inner')
        if dfmt.shape[0] > 0:
            print('******')
            print(mc)
            print(dfmt.shape[0])
            print('******')
            dfmt['Match'] = '-'.join(mc)
            dfm = dfm.append(dfmt[keep_columns + ['Match']].reset_index(drop=True))
            df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
            df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]
    print(dfm.shape[0], df1.shape[0], df2.shape[0])
    if return_unmatched:
        return (dfm.reset_index(drop=True), df1, df2)
    else:
        return dfm.reset_index(drop=True)

In [8]:
def merge_datasets(df1, df2, keep_columns,
                   custom_matches = [], return_unmatched = True, name_changes=True):
    df1 = df1.dropna(axis=1, how='all')
    df2 = df2.dropna(axis=1, how='all')
    
    if "Birth.Year" not in intersect(df1.columns, df2.columns):
        add_cols = ["F4FN", "F4LN", "BY_to_CA", "Current.Age"]
    else:
        add_cols = ["F4FN", "F4LN"]
        
    df1 = add_columns(df1, add_cols)
    df2 = add_columns(df2, add_cols)
    
    
    cols = intersect(df1.columns, df2.columns)
    
    df1 = df1[[col for col in df1.columns
               if col in cols or col == keep_columns[0]]]
    df2 = df2[[col for col in df2.columns
               if col in cols or col == keep_columns[1]]]
    
    base_lists = [
        ['Current.Star', 'Star1', 'Star2', 'Star3', 'Star4', 'Star5','Star6', 'Star7', 'Star8', 'Star9', 'Star10'],
        ['First.Name', 'F4FN'], ['Last.Name', 'F4LN'], ['Appointed.Date'],
        ['Birth.Year', 'Current.Age', 'Current.Age.p1', 'Current.Age.m1', ''],
        ['Middle.Initial', ''], ['Gender', ''], ['Race', ''], ['Suffix.Name', '']
    ]
    
    on_lists = generate_on_lists(cols, base_lists)
    
    if custom_matches:
        on_lists.append(custom_matches)
    
    if name_changes:
        nc_lists = generate_on_lists(cols, [ml for ml in base_lists if "Last.Name" not in ml])
        nc_lists = [nc_list for nc_list in nc_lists if len(nc_list) > 3]
        on_lists.extend(nc_lists)
    
    merged_data = loop_merge(df1, df2, on_lists=on_lists, keep_columns=keep_columns, return_unmatched=return_unmatched)
        
    return(merged_data)

In [9]:
def append_to_reference(df1, df2, keep_columns, custom_matches = [], return_unmatched=False, name_changes=True):
    ml = merge_datasets(df1, df2, keep_columns=keep_columns, 
                        custom_matches=custom_matches, name_changes=name_changes)

    ref = pd.concat([ml[0][keep_columns],
                    ml[1][[keep_columns[0]]],
                    ml[2][[keep_columns[1]]]])[
        keep_columns].reset_index(drop=True)

    if "UID" not in ref.columns:
        ref.insert(0, 'UID', ref.index + 1)
    else:
        ref = ref.sort_values('UID', na_position='last')
        ref['UID'] = ref.index + 1
    df1 = df1.merge(ref, on=keep_columns[0], how='left')
    df2 = df2.merge(ref, on=keep_columns[1], how='left')

    ref = pd.concat([df1, df2]).reset_index(drop=True)
    if return_unmatched:
        return (ref, ml[1], ml[2])
    else:
        return ref

In [10]:
t1 = append_to_reference(asd, asud, ["all_sworn_ID", "all_sworn_units_ID"])

******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Birth.Year', 'Appointed.Date', 'Last.Name', 'First.Name']
32006
******


/Users/invinst/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:1457: RuntimeWarning: divide by zero encountered in long_scalars
  stride //= shape[i]


32006 0 0


In [11]:
ref = t1
profile_cols = ["First.Name", "Last.Name", "Middle.Initial", "Suffix.Name",
                "Appointed.Date", "Birth.Year", "Current.Unit", "Race", "Gender",
               "all_sworn_ID","all_sworn_units_ID"]
mode_cols = profile_cols
profiles = aggregate_data(ref, "UID", mode_cols = mode_cols)
print(profiles.shape)

(32006, 12)


In [12]:
amd = pd.read_csv("input/all-members_demographics.csv.gz")
amd.head()

all_members_ID First.Name Last.Name Suffix.Name Appointed.Date  Birth.Year  \
0           12696    JEFFERY     AARON         NaN     2005-09-26        1971   
1           16366     KARINA     AARON         NaN     2005-09-26        1980   
2            4746     DANIEL     ABATE         NaN     1970-06-15        1942   
3            1201    ANTHONY    ABBATE         NaN     1994-12-05        1968   
4            3122     CARMEL    ABBATE         NaN     1969-01-06        1942   

   Gender      Race Middle.Initial  
0    MALE     WHITE              M  
1  FEMALE  HISPANIC            NaN  
2    MALE     WHITE              P  
3    MALE     WHITE              G  
4    MALE     WHITE              G

In [13]:
profiles.head()

UID First.Name Last.Name Middle.Initial Suffix.Name Appointed.Date  \
0  25766    JEFFERY     AARON              M         NaN     2005-09-26   
1   9516     KARINA     AARON            NaN         NaN     2005-09-26   
2  26592     DANIEL     ABATE              P         NaN     1970-06-15   
3  20300    ANTHONY    ABBATE              G         NaN     1994-12-05   
4  20068     CARMEL    ABBATE              G         NaN     1969-01-06   

   Birth.Year  Current.Unit      Race  Gender all_sworn_ID all_sworn_units_ID  
0        1971           3.0     WHITE    MALE        12735              12735  
1        1980          15.0  HISPANIC  FEMALE        16413              16413  
2        1942         543.0     WHITE    MALE         4767               4767  
3        1968          20.0     WHITE    MALE         1207               1207  
4        1942         640.0     WHITE    MALE         3134               3134

In [14]:
t2 = append_to_reference(profiles, amd, ["UID", "all_members_ID"],return_unmatched=False)

******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Birth.Year', 'Appointed.Date', 'Last.Name', 'First.Name']
31911
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Birth.Year', 'Appointed.Date', 'F4LN', 'First.Name']
3
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Appointed.Date', 'Last.Name', 'First.Name']
1
******
******
['Suffix.Name', 'Gender', 'Middle.Initial', 'Birth.Year', 'Appointed.Date', 'Last.Name', 'First.Name']
1
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Birth.Year', 'Appointed.Date', 'First.Name']
8
******
31924 82 1


In [16]:
profile_cols = ["First.Name", "Last.Name", "Middle.Initial", "Suffix.Name",
                "Appointed.Date", "Birth.Year", "Current.Unit", "Race", "Gender",
               "all_sworn_ID","all_sworn_units_ID", "all_members_ID"]
mode_cols = profile_cols

In [17]:
print(mode_cols)
t2.head()

['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Birth.Year', 'Current.Unit', 'Race', 'Gender', 'all_sworn_ID', 'all_sworn_units_ID', 'all_members_ID']


Appointed.Date  Birth.Year  Current.Unit First.Name  Gender Last.Name  \
0     2005-09-26        1971           3.0    JEFFERY    MALE     AARON   
1     2005-09-26        1980          15.0     KARINA  FEMALE     AARON   
2     1970-06-15        1942         543.0     DANIEL    MALE     ABATE   
3     1994-12-05        1968          20.0    ANTHONY    MALE    ABBATE   
4     1969-01-06        1942         640.0     CARMEL    MALE    ABBATE   

  Middle.Initial      Race Suffix.Name    UID all_members_ID all_sworn_ID  \
0              M     WHITE         NaN  25766          24612        12735   
1            NaN  HISPANIC         NaN   9516          13138        16413   
2              P     WHITE         NaN  26592          11291         4767   
3              G     WHITE         NaN  20300          19792         1207   
4              G     WHITE         NaN  20068          23136         3134   

  all_sworn_units_ID  
0              12735  
1              16413  
2               4767  
3               1207  
4               3134

In [18]:
profiles = aggregate_data(t2, "UID", mode_cols = mode_cols)

First.Name


/Users/invinst/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Last.Name
Middle.Initial
Suffix.Name
Appointed.Date
Birth.Year
Race
Gender


In [21]:
profiles.head()

UID First.Name Last.Name Middle.Initial Suffix.Name Appointed.Date  \
0  25766    JEFFERY     AARON              M         NaN     2000-02-28   
1   9516    JESSICA     AARON            NaN         NaN     2005-09-26   
2  26592     DANIEL     ABATE              P         NaN     1970-06-15   
3  20300    ANTHONY    ABBATE              G         NaN     1994-12-05   
4  20068     CARMEL    ABBATE              G         NaN     1969-01-06   

   Birth.Year  Current.Unit      Race  Gender all_sworn_ID all_sworn_units_ID  \
0        1971           3.0     WHITE    MALE        12735              12735   
1        1980          15.0  HISPANIC  FEMALE        16413              16413   
2        1942         543.0     WHITE    MALE         4767               4767   
3        1968          20.0     WHITE    MALE         1207               1207   
4        1942         640.0     WHITE    MALE         3134               3134   

  all_members_ID  
0          24612  
1          13138  
2          11291  
3          19792  
4          23136

In [23]:
t2[t2['UID']==9516]

Appointed.Date  Birth.Year  Current.Unit First.Name  Gender Last.Name  \
1         2005-09-26        1980          15.0     KARINA  FEMALE     AARON   
59167     2007-08-27        1984           NaN    JESSICA  FEMALE      SOSA   

      Middle.Initial      Race Suffix.Name   UID all_members_ID all_sworn_ID  \
1                NaN  HISPANIC         NaN  9516          13138        16413   
59167            NaN  HISPANIC         NaN  9516          13138          NaN   

      all_sworn_units_ID  
1                  16413  
59167                NaN

In [59]:
uhd = pd.read_csv("input/unit-history_demographics.csv.gz")

In [60]:
uhd.columns

Index(['unit_history_ID', 'First.Name', 'Last.Name', 'Middle.Initial',
       'Suffix.Name', 'Appointed.Date', 'Current.Age', 'Gender', 'Race',
       'Star1', 'Star2', 'Star3', 'Star4', 'Star5', 'Star6', 'Star7', 'Star8',
       'Star9', 'Star10', 'Current.Unit'],
      dtype='object')

In [62]:
t3 = append_to_reference(profiles, uhd, ["UID", "unit_history_ID"],return_unmatched=True)

******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Current.Age.p1', 'Appointed.Date', 'Last.Name', 'First.Name']
242
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Current.Age.p1', 'Appointed.Date', 'F4LN', 'First.Name']
1
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Current.Age.m1', 'Appointed.Date', 'Last.Name', 'First.Name']
758
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Current.Age.m1', 'Appointed.Date', 'Last.Name', 'F4FN']
1
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Appointed.Date', 'Last.Name', 'First.Name']
267
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Appointed.Date', 'Last.Name', 'F4FN']
1
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Appointed.Date', 'F4LN', 'First.Name']
2
******
******
['Suffix.Name', 'Race', 'Gender', 'Current.Age.p1', 'Appointed.Date', 'Last.Name', 'First.Name']
1121
******
******
['Suffix.Name', 'Race', 'Gende

In [63]:
t3[2]

unit_history_ID First.Name  Last.Name Middle.Initial Suffix.Name  \
0                    1    JEFFERY      AARON            NaN         NaN   
1                    2     KARINA      AARON            NaN         NaN   
6                    7      TERRY     ABBATE            NaN         NaN   
8                    9       LEON      ABBEY            NaN         NaN   
13                  14      LAURA     ABBOTT            NaN         NaN   
14                  15       ZAID   ABDALLAH            NaN         NaN   
19                  20  RESTITUTO    ABEJERO            NaN         NaN   
20                  21    KENNETH      ABELS            NaN         NaN   
23                  24    PATRICK      ABNEY            NaN         NaN   
26                  27      NANCY    ABRAHAM            NaN         NaN   
27                  28       PAUL      ABRAM            NaN         NaN   
30                  31  ALEXANDER  ABRASEVIC            NaN         NaN   
31                  32    RICARDO      ABREU            NaN         NaN   
34                  35      ELIAS   ABUDAYEH            NaN         NaN   
35                  36   MOHAMMAD   ABURIZEG            NaN         NaN   
37                  38   JENNIFER    ACCARDO            NaN         NaN   
38                  39   ROSEMARY    ACCARDO            NaN         NaN   
39                  40     THOMAS    ACCARDO            NaN         NaN   
40                  41      ROCCO  ACCETTURA            NaN         NaN   
45                  46     EDWARD    ACEVEDO            NaN         NaN   
46                  47     EDWARD    ACEVEDO            NaN         NaN   
47                  48       ERIC    ACEVEDO            NaN         NaN   
48                  49       ILIA    ACEVEDO            NaN         NaN   
50                  51      JASON    ACEVEDO            NaN         NaN   
51                  52       JEFF    ACEVEDO            NaN         NaN   
56                  57     NADINE    ACEVEDO            NaN         NaN   
57                  58      PEDRO    ACEVEDO            NaN         NaN   
59                  60      SALLY    ACEVEDO            NaN         NaN   
60                  61       JUAN     ACEVES            NaN         NaN   
62                  63   CLEMENTE     ACEVEZ            NaN         NaN   
...                ...        ...        ...            ...         ...   
31544            31545     SAMUEL     ZROSKI            NaN         NaN   
31545            31546    SHANNON       ZUBB            NaN         NaN   
31546            31547       GINA     ZUBECK            NaN         NaN   
31547            31548      JAMES     ZUBECK            NaN         NaN   
31548            31549    MICHAEL      ZUBER            NaN         NaN   
31549            31550   PATRICIA      ZUBER            NaN         NaN   
31551            31552   THEODORE      ZUDYK            NaN         NaN   
31555            31556   NICHOLAS     ZUKLIC            NaN         NaN   
31556            31557      FRANK    ZUKOSKI            NaN          JR   
31558            31559    RICHARD      ZULEY            NaN         NaN   
31560            31561     JOSEPH     ZULKEY            NaN         NaN   
31562            31563       JOHN   ZUMBROCK            NaN         NaN   
31563            31564      CHRIS     ZUNIGA            NaN         NaN   
31568            31569      OSCAR     ZUNIGA            NaN         NaN   
31569            31570     RONALD     ZUNIGA            NaN         NaN   
31570            31571      KELLY   ZUPANCIC            NaN         NaN   
31571            31572    MICHAEL   ZUPANCIC            NaN         NaN   
31574            31575    DEBORAH    ZURAWIK            NaN         NaN   
31575            31576      JAMES    ZURAWIK            NaN         NaN   
31576            31577      JAMES   ZURAWSKI            NaN         NaN   
31577            31578      JAMES   ZURAWSKI            NaN         NaN   
31579            31580    GREGOR

In [64]:
t3[1]

UID First.Name     Last.Name Middle.Initial Suffix.Name  \
0      25766    JEFFERY         AARON              M         NaN   
1       9516    JESSICA         AARON            NaN         NaN   
6      25772    MICHAEL        ABBATE              M         NaN   
7       4232      DAVID         ABBEY              B         NaN   
8       7666       LEON         ABBEY            NaN         NaN   
13     13437      LAURA        ABBOTT              F         NaN   
19      1624  RESTITUTO       ABEJERO              E         NaN   
20     24596       JOHN         ABELS              L         NaN   
27      4185    ANTHONY         ABRAM              A         NaN   
28      7500     GEORGE         ABRAM              W         NaN   
30     15521       FRED         ABRAN              A         NaN   
31     31443  ALEXANDER     ABRASEVIC            NaN         NaN   
32     11396     DANIEL         ABREU            NaN         NaN   
35     26000      ELIAS      ABUDAYEH              M         NaN   
38     29536    ANTHONY       ACCARDO              V         NaN   
39     13044   JENNIFER       ACCARDO              A         NaN   
40     15053      KARIN       ACCARDO            NaN         NaN   
43     10309      AARON       ACEVEDO              F         NaN   
47     10941      DAVID       ACEVEDO              M         NaN   
48     10500     EDWARD       ACEVEDO              J         NaN   
49     12145     EDWARD       ACEVEDO            NaN         NaN   
50      9905       ERIC       ACEVEDO              A         NaN   
51      9438       ILIA       ACEVEDO            NaN         NaN   
54     11637    ADNARDO       ACEVEDO            NaN         NaN   
55     10271     EMILIO       ACEVEDO              F         NaN   
62      9295    REBECCA       ACEVEDO            NaN         NaN   
63     10079       JUAN        ACEVES              C         NaN   
65     11062   CLEMENTE        ACEVEZ              R         NaN   
66     14818       LISA       ACHILLY              S         NaN   
68      9989      JAIME        ACOSTA              A         NaN   
...      ...        ...           ...            ...         ...   
31954  28058      JAMES        GLENKE              R         NaN   
31955  14679  JACQUELIN         REGAN              O         NaN   
31957  29424     PHILIP       FERRARO              V         NaN   
31962   1103      FRANK      BORECZKY              J          JR   
31964  26686    MICHAEL         KELLY              P         NaN   
31966  29531     JOSEPH       PALUMBO              V         NaN   
31967  20214    GREGORY         ALLEN              G         NaN   
31969  11278      CHRIS         GARAY              V         NaN   
31970  10067       ERIC        VALDEZ              C         NaN   
31972  10836      JAIME         GOVEA              M         NaN   
31974  12543        NOE     PEMBERTON              R         NaN   
31975  12449      DAVID      REGALADO            NaN         NaN   
31976  11842  ALEXANDER        CEREZO            NaN         NaN   
31977  14055      KELLY      SATRIANO              L         NaN   
31981  14201    DEBORAH     LATKOWSKI              M         NaN   
31982  18812     ANDREW       WALLACE              E         NaN   
31983  20152      JAMES    CUNNINGHAM              G         NaN   
31984  21497     GEORGE       SWAGLER              J         NaN   
31985  13345    BRIDGET      BRUBAKER              E         NaN   
31987   9630     ADRIEL         SERPE                        NaN   
31990  28384      HENRY        BARNAS              S         NaN   
31991  27875      JAMES         WEEKS              R         NaN   
31993  13917     BRENDA       GATRELL              L         NaN   
31994  20746       ADAM        HANLON              J         NaN   
31995  15449    FRANCIS          MACK              A         NaN   
31996  26112    MATEUSZ      URBANIAK              M         NaN   
31997  24845      BRYAN        MURPHY              L         NaN 